In [ ]:
from research.weight_estimation.akpd_utils.akpd import AKPD
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
import json
import os
import  pandas as pd
import numpy as np
import matplotlib.patches as patches
from tqdm import tqdm
from utils import utils, data_prep, sector
import cv2
from PIL import Image


import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
import importlib
importlib.reload(utils)

In [ ]:

LICE_BBOX_COLOR = ['b', 'r'] # bbox edge color
LICE_CATEGORY = ['ADULT_FEMALE', 'MOVING']

# load annotation data

In [ ]:
annotation_data_akpd  = pd.read_pickle("annotation_data_akpd.pkl")
annotation_data_akpd['left_kps'] = annotation_data_akpd['left_kps'].apply(lambda x: x[0]["leftCrop"])


In [ ]:
annotation_data_akpd.shape

In [ ]:
annotation_data_akpd.head(1)

# visualize key points

In [ ]:
for idx, sf in tqdm(annotation_data_akpd.iterrows()):
    if idx < 500:
        continue
    if idx > 504:
        break
    left_image_f, bucket, left_image_key = s3_access_utils.download_from_url(sf["left_crop_url"])
    image_f = left_image_f 

        
    fig, ax = plt.subplots(figsize=(10, 10))
    
    
    img = Image.open(image_f)

    alpha = 3 # Contrast control (1.0-3.0)
    beta = 20 # Brightness control (0-100)

    img = np.asarray(img)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    ax.imshow(adjusted)

        
    for kp in sf.left_kps:
        x, y = kp['xCrop'], kp['yCrop']
        bp = kp['keypointType']
        ax.scatter(x, y, c='red', marker='o')
        ax.annotate(bp, (x, y), color='red')
    plt.show()

# summary stats of lice wrt AKPD

In [ ]:
def distance(p1, p2, metrics = "euclidean"):
    assert metrics in ("euclidean", "x", "y"), "wrong metrics"
    dx, dy = abs(p1[0] - p2[0]), abs(p1[1] - p2[1])
    if metrics == "euclidean": 
        return pow(pow(dx, 2) + pow(dy, 2), 0.5)
    elif metrics == "x": 
        return dx
    else:
        return dy

In [ ]:
def get_neighbor_keypoints(lice, left_kps, metrics = "euclidean"):
    lice_xy = lice['position']['left'], lice['position']['top']
    neighbors = {}
    for kp in left_kps:
        kp_xy = kp['xCrop'], kp['yCrop']
        neighbors[kp['keypointType']] = distance(lice_xy, kp_xy, metrics)
    return {k: v for k, v in sorted(neighbors.items(), key=lambda item: item[1])}




In [ ]:
def get_crop(kps, keypointType1, keypointType2, k):
    p1 = np.array(get_kp_location(kps, keypointType1))
    p2 = np.array(get_kp_location(kps, keypointType2))
    k = np.array(k)
    return utils.xyxy2xywh(p1, p2, k)


def add_get_crop(df):
    df['adipose_tail_crop'] = df.apply(lambda fish: 
                                       get_crop(fish.left_kps, 
                                                "ADIPOSE_FIN", "TAIL_NOTCH", 
                                                np.array([[0, -0.2], [0.5, 1]])), 
                                       axis = 1)
    df['adipose_dorsal_crop'] = df.apply(lambda fish: 
                                         get_crop(fish.left_kps, 
                                                  "ADIPOSE_FIN", "DORSAL_FIN", 
                                                  np.array([[0, -0.2], [0.7, 1]])), 
                                         axis = 1)
    df['anal_tail_crop'] = df.apply(lambda fish: 
                                    get_crop(fish.left_kps, 
                                             "ANAL_FIN", "TAIL_NOTCH", 
                                             np.array([[0, -0.2], [0.8, 1]])), 
                                    axis = 1)
    df['pectoral_eye_crop'] = df.apply(lambda fish: 
                                       get_crop(fish.left_kps, 
                                                "PECTORAL_FIN", "EYE", 
                                                np.array([[0, -0.3], [1, 2.3]])), 
                                       axis = 1)
    return df
    
    

In [ ]:
annotation_data_akpd = add_get_crop(annotation_data_akpd)



annotation_data_akpd.head(1)

In [ ]:
lice_data = pd.DataFrame()
for idx, sf in tqdm(annotation_data_akpd.iterrows()):
#     if idx > 10:
#         break
    if sf['annotation']:
        adipose_tail_crop = sf.adipose_tail_crop
        adipose_dorsal_crop = sf.adipose_dorsal_crop
        pectoral_eye_crop = sf.pectoral_eye_crop
        anal_tail_crop = sf.anal_tail_crop
        for lice in sf['annotation']:
            if lice['category'] not in ['ADULT_FEMALE', 'MOVING']:
                continue            
            lp = lice['position'] #lice position

            x, y, w, h = lp["left"], lp["top"], lp["width"], lp["height"]
            lice_xywh = x, y, w, h
            
            in_crop = None
            if data_prep.is_in_crop(lice_xywh, adipose_tail_crop):
                in_crop = "adipose_tail_crop"
            elif data_prep.is_in_crop(lice_xywh, adipose_dorsal_crop):
                in_crop = "adipose_dorsal_crop"
            elif data_prep.is_in_crop(lice_xywh, pectoral_eye_crop):
                in_crop = "pectoral_eye_crop"
            elif data_prep.is_in_crop(lice_xywh, anal_tail_crop):
                in_crop = "anal_tail_crop"
                
            lice_data = lice_data.append({"category": lice['category'],
                            "fish_image_url": sf['left_crop_url'],
                            "location": lice['location'],
                            "left": x,
                            "top": y,
                            "width": w, 
                            "height": h,
                            "neighbors_euclidean": list(get_neighbor_keypoints(lice, sf.left_kps, 'euclidean'))[0],
                            "neighbors_x": list(get_neighbor_keypoints(lice, sf.left_kps, 'x'))[0],
                            "neighbors_y": list(get_neighbor_keypoints(lice, sf.left_kps, 'y'))[0],
                            "fish_image_width": sf['left_crop_metadata']['width'],
                            "fish_image_height": sf['left_crop_metadata']['height'],
                            "left_kps": sf["left_kps"],
                            "adipose_tail_crop": sf['adipose_tail_crop'],
                            "adipose_dorsal_crop": sf['adipose_dorsal_crop'],
                            "anal_tail_crop": sf['anal_tail_crop'],
                            "pectoral_eye_crop": sf['pectoral_eye_crop'],
                            "in_crop": in_crop
                           }, ignore_index=True)

In [ ]:
lice_data.shape

In [ ]:
def get_kp_location(kps, keypointType):
    "return kps x, y from left_kps"
    for kp in kps:
        if kp["keypointType"] == keypointType:
            return np.array((kp['xCrop'], kp['yCrop']))

        

In [ ]:
def get_proportion(lice_xy, kps, keypointType1, keypointType2):

    p1 = np.array(get_kp_location(kps, keypointType1))
    p2 = np.array(get_kp_location(kps, keypointType2))
    lice_xy = np.array(lice_xy)
    try:
        return (p1 - lice_xy) / (p1 - p2) 
    except ZeroDivisionError:
        return None

In [ ]:
def add_proportion(df):
    df['anal_tail_proportion'] = df.apply(lambda lice: get_proportion((lice.left, lice.top), lice.left_kps, "ANAL_FIN", "TAIL_NOTCH"), axis=1)
    df['adipose_tail_proportion'] = df.apply(lambda lice: get_proportion((lice.left, lice.top), lice.left_kps, "ADIPOSE_FIN", "TAIL_NOTCH"), axis=1)
    return df



In [ ]:
lice_data = add_proportion(lice_data)

In [ ]:
lice_data.head(1)

In [ ]:
subdata = lice_data[(lice_data["neighbors_euclidean"] == "ADIPOSE_FIN") 
                    & (lice_data["location"] == "TOP")
                   ]

In [ ]:
subdata.head(1)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 6))

proportion_x = subdata["anal_tail_proportion"].apply(lambda x: x[0])
proportion_y = subdata["anal_tail_proportion"].apply(lambda x: x[1])

proportion_x = proportion_x[~np.isinf(proportion_x)]
proportion_y = proportion_y[~np.isinf(proportion_y)]

x_cutoff = np.percentile(proportion_x, (1, 99))
y_cutoff = np.percentile(proportion_y, (1, 99))
print(x_cutoff)
print(y_cutoff)

ax[0].hist(proportion_x, bins = 100, range = x_cutoff)
ax[1].hist(proportion_y, bins = 100, range = y_cutoff)
plt.show()

In [ ]:
print(np.percentile(proportion_x, (1, 99)))
print(np.percentile(proportion_y, (1, 99)))

In [ ]:
k_between = np.array([0.5, 1.3])
k_outside = np.array([-1.2, -2])

In [ ]:
np.average(np.array([k_between,k_outside]), axis = 0)

# dev

In [ ]:
def get_adipose_crop(kps, k):
    AF = np.array(get_kp_location(kps, "ADIPOSE_FIN"))
    TN = np.array(get_kp_location(kps, "TAIL_NOTCH"))
    k = np.array(k)
    return AF - k * (AF - TN)

def add_adipose_crop(df):
    df['adipose_crop_between'] = df.apply(lambda fish: get_adipose_crop(fish.left_kps, k_between), axis=1)
    df['adipose_crop_outside'] = df.apply(lambda fish: get_adipose_crop(fish.left_kps, k_outside), axis=1)
    return df


In [ ]:
annotation_data_akpd = add_adipose_crop(annotation_data_akpd)

In [ ]:
sum(lice_data.in_crop.apply(lambda x: x is None))

# plot

In [ ]:
lice_data.groupby(['category']).neighbors_euclidean.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True, 
                                                               color=LICE_BBOX_COLOR, 
                                                               title = "Bar Chart of Lice category by closest key apoints")

lice_data.groupby(['neighbors_euclidean']).category.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,  
                                                               title = "Bar Chart of Lice category by closest key apoints")



In [ ]:
lice_data.groupby('category').in_crop.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True, 
                                                               color=LICE_BBOX_COLOR, 
                                                               title = "Bar Chart of Lice by closet key apoints")



In [ ]:
lice_data.groupby('location').in_crop.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,  
                                                               title = "Bar Chart of Lice by closet key apoints")


In [ ]:
lice_data.groupby('location').neighbors_euclidean.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,  
                                                               title = "Bar Chart of Lice location by closest key apoints")


In [ ]:
lice_data.groupby('location').neighbors_euclidean.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,  
                                                               title = "Bar Chart of Lice by closet key apoints")



In [ ]:
lice_data.groupby('neighbors_euclidean').location.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,
                                                               title = "Bar Chart of Closest Key Point by Location")


lice_data[lice_data.category == 'ADULT_FEMALE'].groupby('neighbors_euclidean').location.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,
                                                               title = "Bar Chart of Lice by Location")



lice_data[lice_data.category == 'MOVING'].groupby('location').neighbors_euclidean.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,
                                                               title = "Bar Chart of Lice by Location")



lice_data[lice_data.category == 'ADULT_FEMALE'].groupby('location').neighbors_euclidean.value_counts().unstack(0).plot(kind='barh', 
                                                               legend=True,
                                                               title = "Bar Chart of Lice by Location")




# calculate sector

In [ ]:
def face_left(kps):
    ul = get_kp_location(kps, "UPPER_LIP")
    tn = get_kp_location(kps, "TAIL_NOTCH")
    return ul[0] < tn[0]

In [ ]:
def is_between_v(p, a, b):
    """
    Check whether vector p is between vector a and b
    check if cross product/sin(theta) has the same singe:
        a X b * a X p >= 0 and b X a * b X p >= 0
    p, a, b: np.array with len 2
    
    """

    return (((a[1] * b[0] - a[0] * b[1]) * (a[1] * p[0] - a[0] * p[1]) >= 0) 
           and ((b[1] * a[0] - b[0] * a[1]) * (b[1] * p[0] - b[0] * p[1]) >= 0))


def is_between_p(P, A, B, O):
    """
    Check whether point P is between point A and B with O as origin
    P, A, B, O: np array with len 2
    """
    a = A - O
    b = B - O
    p = P - O

    return is_between_v(p, a, b)

In [ ]:
def get_auxiliary_kps(kps):
  
    ad_fin = get_kp_location(kps, "ADIPOSE_FIN")
    an_fin = get_kp_location(kps, "ANAL_FIN")
    ad_an_mid = np.average(np.array([ad_fin, an_fin]), axis = 0)

    ds_fin = get_kp_location(kps, "DORSAL_FIN")
    pv_fin = get_kp_location(kps, "PELVIC_FIN")
    ds_pv_mid = np.average(np.array([ds_fin, pv_fin]), axis = 0)

    pt_fin = get_kp_location(kps, "PECTORAL_FIN")


    h1 = 0.25 * (pv_fin - pt_fin) + pt_fin
    h0 = h1 + 0.7 * (ds_fin - pv_fin)
    
    h_mid = np.average(np.array([h0, h1]), axis = 0)
    
    return {"ad_an_mid": ad_an_mid, "ds_pv_mid": ds_pv_mid, "h0": h0, "h1": h1, "h_mid": h_mid}

In [ ]:
# in_dorsal_back = lambda p: is_between_p(p, ad_fin, tn, ad_an_mid)
# in_ventral_back = lambda p: is_between_p(p, an_fin, tn, ad_an_mid)

# in_dorsal_mid = lambda p: is_between_p(p, ds_fin, ad_an_mid, ds_pv_mid)
# in_ventral_mid = lambda p: is_between_p(p, pv_fin, ad_an_mid, ds_pv_mid)

# in_dorsal_front = lambda p: is_between_p(p, h0, ds_pv_mid, h_mid)
# in_ventral_front = lambda p: is_between_p(p, h1, ds_pv_mid, h_mid)

# in_head = lambda p: is_between_p(p, h0, eye, h_mid) and is_between_p(p, h1, eye, h_mid)


In [ ]:
def get_sector(p, kps):
    eye = get_kp_location(kps, "EYE")
    tn = get_kp_location(kps, "TAIL_NOTCH")
    ad_fin = get_kp_location(kps, "ADIPOSE_FIN")
    an_fin = get_kp_location(kps, "ANAL_FIN")
    ds_fin = get_kp_location(kps, "DORSAL_FIN")
    pv_fin = get_kp_location(kps, "PELVIC_FIN")
    pt_fin = get_kp_location(kps, "PECTORAL_FIN")

    aux_kps = get_auxiliary_kps(kps)
    
    ad_an_mid = aux_kps["ad_an_mid"]
    ds_pv_mid = aux_kps["ds_pv_mid"]
    h1 = aux_kps["h1"]
    h0 = aux_kps["h0"]
    h_mid = aux_kps["h_mid"]

    if is_between_p(p, ad_fin, tn, ad_an_mid): return "dorsal_back"
    elif is_between_p(p, an_fin, tn, ad_an_mid): return "ventral_back"
    elif is_between_p(p, ds_fin, ad_an_mid, ds_pv_mid): return "dorsal_mid"
    elif is_between_p(p, pv_fin, ad_an_mid, ds_pv_mid): return "ventral_mid"
    elif is_between_p(p, h0, ds_pv_mid, h_mid): return "dorsal_front"
    elif is_between_p(p, h1, ds_pv_mid, h_mid): return "ventral_front"
    else: return "head"

# Unit test

In [ ]:
# True
A = np.array([5, 1])
B = np.array([1, 4])
P = np.array([3, 2])
O = np.array([0, 0])
is_between_p(P, A, B, O)

In [ ]:
# False
A = np.array([5, 1])
B = np.array([1, 4])
P = np.array([3, -2])
O = np.array([0, 0])
is_between_p(P, A, B, O)

In [ ]:
# True
A = np.array([1, -4])
B = np.array([1, 4])
P = np.array([5, -2])
O = np.array([0, 0])
is_between_p(P, A, B, O)

# line segment

In [ ]:
# 
num_image = 0
for idx, sf in tqdm(annotation_data_akpd.iloc[:100].iterrows()):

    if not sf.annotation or not face_left(sf.left_kps):
        continue
    num_image += 1
    if num_image > 15: break
    left_image_f, bucket, left_image_key = s3_access_utils.download_from_url(sf["left_crop_url"])
    image_f = left_image_f 

        
    fig, ax = plt.subplots(figsize=(10, 10))
    
    
    img = Image.open(image_f)

    alpha = 3 # Contrast control (1.0-3.0)
    beta = 20 # Brightness control (0-100)

    img = np.asarray(img)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    ax.imshow(adjusted)


    for kp in sf.left_kps:
        k1, k2 = kp['xCrop'], kp['yCrop']
        bp = kp['keypointType']
        ax.scatter(k1, k2, c='orange', marker='o')
        #ax.annotate(bp, (k1, k2), color='orange')
    kps = sf.left_kps
    eye = get_kp_location(kps, "EYE")
    tn = get_kp_location(kps, "TAIL_NOTCH")
    ad_fin = get_kp_location(kps, "ADIPOSE_FIN")
    an_fin = get_kp_location(kps, "ANAL_FIN")
    ds_fin = get_kp_location(kps, "DORSAL_FIN")
    pv_fin = get_kp_location(kps, "PELVIC_FIN")
    pt_fin = get_kp_location(kps, "PECTORAL_FIN")

    aux_kps = get_auxiliary_kps(kps)
    
    ad_an_mid = aux_kps["ad_an_mid"]
    ds_pv_mid = aux_kps["ds_pv_mid"]
    h1 = aux_kps["h1"]
    h0 = aux_kps["h0"]
    h_mid = aux_kps["h_mid"]
    plt.plot([ad_an_mid[0], tn[0]],[ad_an_mid[1], tn[1]],'k-')
    plt.plot([ad_an_mid[0], ds_pv_mid[0]],[ad_an_mid[1], ds_pv_mid[1]],'k-')
    plt.plot([ds_pv_mid[0], h_mid[0]],[ds_pv_mid[1], h_mid[1]],'k-')
    
    plt.plot([ad_fin[0], an_fin[0]],[ad_fin[1], an_fin[1]],'k-')    
    plt.plot([ds_fin[0], pv_fin[0]],[ds_fin[1], pv_fin[1]],'k-')
    plt.plot([h0[0], h1[0]], [h0[1], h1[1]],'k-')

    for lice in sf['annotation']:
        lp = lice['position'] 
        x, y, w, h = lp["left"], lp["top"], lp["width"], lp["height"]
        class_index = LICE_CATEGORY.index(lice['category'])
        ec = LICE_BBOX_COLOR[class_index]
        rect = patches.Rectangle((x, y), w, h,linewidth=1,edgecolor=ec,facecolor='none') 
        ax.add_patch(rect)
        sector_label = get_sector(np.array([x, y]), sf.left_kps)
        ax.annotate(sector_label, (x, y), color = ec)
    plt.show()

In [ ]:
#fish 



nrows = 3
figure, axes = plt.subplots(nrows=nrows, ncols=2, figsize=(20, nrows * 6))

        
        
num_pic = -1
for idx, row in tqdm(annotation_data_akpd.iterrows()):
    if not row['annotation'] or idx < 22:
        continue
    
    num_pic += 1
    if num_pic >= nrows * 2:
        break
    
    left_image_f, bucket, left_image_key = s3_access_utils.download_from_url(row.left_crop_url)
    image_f = left_image_f         
    img = Image.open(image_f)

    alpha = 3 # Contrast control (1.0-3.0)
    beta = 20 # Brightness control (0-100)

    img = np.asarray(img)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    axes[num_pic // 2, num_pic % 2].imshow(adjusted)




    x, y, w, h = row.adipose_tail_crop
    rect = patches.Rectangle((x, y), w, h,linewidth=3,edgecolor='y',facecolor='none')    
    axes[num_pic // 2, num_pic % 2].add_patch(rect)   

    x, y, w, h = row.anal_tail_crop
    rect = patches.Rectangle((x, y), w, h,linewidth=3,edgecolor='y',facecolor='none')    
    axes[num_pic // 2, num_pic % 2].add_patch(rect)    

    x, y, w, h = row.adipose_dorsal_crop
    rect = patches.Rectangle((x, y), w, h,linewidth=3,edgecolor='y',facecolor='none')    
    axes[num_pic // 2, num_pic % 2].add_patch(rect)  


    x, y, w, h = row.pectoral_eye_crop
    rect = patches.Rectangle((x, y), w, h,linewidth=3,edgecolor='y',facecolor='none')    
    axes[num_pic // 2, num_pic % 2].add_patch(rect)     


    for kp in row.left_kps:
        x, y = kp['xCrop'], kp['yCrop']
        bp = kp['keypointType']
        axes[num_pic // 2, num_pic % 2].scatter(x, y, c='red', marker='o')
        axes[num_pic // 2, num_pic % 2].annotate(bp, (x, y), color='red')

figure.tight_layout()


In [ ]:
#lice


num_pic = 0
for idx, lice in tqdm(lice_data.iterrows()):
    if idx < 150:
        continue
    if not lice.in_crop:
        num_pic += 1
        if num_pic > 10:
            break
        left_image_f, bucket, left_image_key = s3_access_utils.download_from_url(lice.fish_image_url)
        image_f = left_image_f         
        fig, ax = plt.subplots(figsize=(10, 10))


        img = Image.open(image_f)

        alpha = 3 # Contrast control (1.0-3.0)
        beta = 20 # Brightness control (0-100)

        img = np.asarray(img)
        adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        ax.imshow(adjusted)
        
        x, y, w, h = lice.adipose_tail_crop
        rect = patches.Rectangle((x, y), w, h,linewidth=1,edgecolor='y',facecolor='none')    
        ax.add_patch(rect)   
        
        x, y, w, h = lice.anal_tail_crop
        rect = patches.Rectangle((x, y), w, h,linewidth=1,edgecolor='y',facecolor='none')    
        ax.add_patch(rect)    

        x, y, w, h = lice.adipose_dorsal_crop
        rect = patches.Rectangle((x, y), w, h,linewidth=1,edgecolor='y',facecolor='none')    
        ax.add_patch(rect)  
        
        
        x, y, w, h = lice.pectoral_eye_crop
        rect = patches.Rectangle((x, y), w, h,linewidth=1,edgecolor='y',facecolor='none')    
        ax.add_patch(rect)          

        class_index = LICE_CATEGORY.index(lice.category)
        ec = LICE_BBOX_COLOR[class_index]

        
        rect = patches.Rectangle((lice.left, lice.top), lice.width, lice.height,linewidth=1,edgecolor=ec,facecolor='none')    
        ax.add_patch(rect) 

        for kp in lice.left_kps:
            x, y = kp['xCrop'], kp['yCrop']
            bp = kp['keypointType']
            ax.scatter(x, y, c='red', marker='o')
            ax.annotate(bp, (x, y), color='red')
        plt.show()
